# userAuthMethod

### This notebook shows the use of Microsoft Graph PowerShell to locate user who does not have his MFA setup properly. The cmdlet Get-MguserAuthenticationMethod is used. If an user has setup MFA properly, he should have "#microsoft.graph.microsoftAuthenticatorAuthenticationMethod" and/ or "#microsoft.graph.phoneAuthenticationMethod" as return.

In [13]:
connect-mggraph -scopes "UserAuthenticationMethod.Read.All, User.Read.All"

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code IUX3DZQXV to authenticate.
Welcome to Microsoft Graph!

Connected via delegated access using 14d82eec-204b-4c2f-b7e8-296a70dab67e
Readme: https://aka.ms/graph/sdk/powershell
SDK Docs: https://aka.ms/graph/sdk/powershell/docs
API Docs: https://aka.ms/graph/docs

NOTE: You can use the -NoWelcome parameter to suppress this message.



In [14]:
$users = get-mguser -all

In [15]:
$users


DisplayName       Id                                   Mail                              UserPrinci
                                                                                         palName
-----------       --                                   ----                              ----------
Adele Vance       e065d899-0464-4a43-b02a-206028b27982 AdeleV@61b3yr.onmicrosoft.com     AdeleV@61…
Alex Wilber       c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726 AlexW@61b3yr.onmicrosoft.com      AlexW@61b…
Daniel Carmack    9a2e7c71-2c59-4b80-ada4-19d8534e6d99 Daniel.C@61b3yr.onmicrosoft.com   Daniel.C@…
Diego Siciliani   0a02a0f1-48e1-4010-b904-c7650b34c112 DiegoS@61b3yr.onmicrosoft.com     DiegoS@61…
Grady Archie      2ea73151-a011-47b5-9a99-8d37adc221b5 GradyA@61b3yr.onmicrosoft.com     GradyA@61…
Henrietta Mueller 2c758750-2644-4e13-975f-453e0dfa2263 HenriettaM@61b3yr.onmicrosoft.com Henrietta…
Isaiah Langer     f243a1ce-a5e6-42e1-abe1-f49ca450d6a0 IsaiahL@61b3yr.onmicrosoft.com    IsaiahL@6…
Jo

In [16]:
# get all auth methods

$tmpObjArr = @()
$users | ForEach-Object -Process {
                           $methods = get-mguserAuthenticationmethod -userid $_.Id
                           $hasMFASetup = $false
                           
                           foreach ($m in $methods) {
                              $extraProps = $m.ToJsonString() | ConvertFrom-Json
                              $tmpObj = [pscustomobject] @{
                                 DisplayName = $_.DisplayName
                                 AuthMethod = $extraProps."@odata.type"
                              }
                              # $tmpObj
                              $tmpObjArr += $tmpObj
                           } 
                    } `
                    -End { $tmpObjArr | Format-Table }


DisplayName       AuthMethod
-----------       ----------
Adele Vance       #microsoft.graph.passwordAuthenticationMethod
Alex Wilber       #microsoft.graph.passwordAuthenticationMethod
Daniel Carmack    #microsoft.graph.passwordAuthenticationMethod
Diego Siciliani   #microsoft.graph.passwordAuthenticationMethod
Grady Archie      #microsoft.graph.passwordAuthenticationMethod
Henrietta Mueller #microsoft.graph.passwordAuthenticationMethod
Isaiah Langer     #microsoft.graph.passwordAuthenticationMethod
Johanna Lorenz    #microsoft.graph.passwordAuthenticationMethod
Joni Sherman      #microsoft.graph.passwordAuthenticationMethod
Lee Gu            #microsoft.graph.phoneAuthenticationMethod
Lee Gu            #microsoft.graph.passwordAuthenticationMethod
Lee Gu            #microsoft.graph.windowsHelloForBusinessAuthenticationMethod
Lee Gu            #microsoft.graph.windowsHelloForBusinessAuthenticationMethod
Lee Gu            #microsoft.graph.windowsHelloForBusinessAuthenticationMethod
Lee

In [17]:
# filter the $tmpObjArr using regular expressions ".*phone.*", ".*Authenticator.*" and look for users
# without these auth methods

$name = $null
$MFASetup = $false
foreach ($r in $tmpObjArr)
{
   if ($r.DisplayName -ne $name)
   {
      if ($name -ne $null)
      {
         if (!$MFASetup) {
            "{0} has not setup MFA properly" -f $name
         }
      }
      $name = $r.DisplayName
      $MFASetup = $false
   }
   if ($r.AuthMethod -match ".*phone.*" -Or $r.AuthMethod -match ".*Authenticator.*")
   {
      $MFASetup = $true
   }
}

Adele Vance has not setup MFA properly
Alex Wilber has not setup MFA properly
Daniel Carmack has not setup MFA properly
Diego Siciliani has not setup MFA properly
Grady Archie has not setup MFA properly
Henrietta Mueller has not setup MFA properly
Isaiah Langer has not setup MFA properly
Johanna Lorenz has not setup MFA properly
Joni Sherman has not setup MFA properly
Lidia Holloway has not setup MFA properly
Lynne Robbins has not setup MFA properly
Megan Bowen has not setup MFA properly
Miriam Graham has not setup MFA properly
Nestor Wilke has not setup MFA properly
Patti Fernandez has not setup MFA properly


In [18]:
Disconnect-MgGraph


ClientId               : 14d82eec-204b-4c2f-b7e8-296a70dab67e
TenantId               : 387d3aa5-b60a-4c48-9073-210676be29fc
Scopes                 : {AuditLog.Read.All, Group.ReadWrite.All, openid, profile…}
AuthType               : Delegated
TokenCredentialType    : DeviceCode
CertificateThumbprint  : 
CertificateSubjectName : 
Account                : plj611@61b3yr.onmicrosoft.com
AppName                : Microsoft Graph Command Line Tools
ContextScope           : Process
Certificate            : 
PSHostVersion          : 1.0.0
ManagedIdentityId      : 
ClientSecret           : 
Environment            : Global

